In [ ]:
# Завантаження бібліотек
import numpy as np # numpy для математичних обчислень
import pickle # pickle для завантаження збереженої моделі

In [ ]:
# Функція для завантаження збережених параметрів
with open(filename, 'rb') as model_file:
        # Завантаження параметрів моделі з файлу за допомогою pickle
        parameters = pickle.load(model_file)
    return parameters

# Виклик функції load_model_parameters
model_parameters = load_model_parameters('my_model.pkl')

# Витяг параметрів з завантаженого словника
kernel_conv1 = model_parameters['kernel_conv1']
bias_conv1 = model_parameters['bias_conv1']
kernel_conv2 = model_parameters['kernel_conv2']
bias_conv2 = model_parameters['bias_conv2']
kernel_conv3 = model_parameters['kernel_conv3']
bias_conv3 = model_parameters['bias_conv3']
weights_f6 = model_parameters['weights_f6']
bias_f6 = model_parameters['bias_f6']
weights_output = model_parameters['weights_output']
bias_output = model_parameters['bias_output']

In [ ]:
# Функція активації ReLU
def relu(x):
    return np.maximum(0, x)

In [ ]:
# Функція активації Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [ ]:
# Шар згортки
def convolution_layer(input_value, kernel, bias, stride=1, padding=0):
    # Визначення розмірів вхідного значення
    input_depth, input_height, input_width = input_value.shape
    # Визначення розмірів ядра
    kernel_depth, kernel_height, kernel_width = kernel.shape
    # Розрахунок висоти та ширини вихідного значення
    output_height = (input_height - kernel_height + 2 * padding) // stride + 1
    output_width = (input_width - kernel_width + 2 * padding) // stride + 1
    # Ініціалізація вихідного значення нулями
    output_value = np.zeros((kernel_depth, output_height, output_width))
    # Додавання паддінгу до вхідного значення
    padded_input = np.pad(input_value, ((0, 0), (padding, padding), (padding, padding)), mode='constant')

    # Перебір по всіх глибинах ядра
    for d in range(kernel_depth):
        # Перебір по висоті вхідного значення з урахуванням кроку
        for i in range(0, input_height - kernel_height + 1, stride):
            # Перебір по ширині вхідного значення з урахуванням кроку
            for j in range(0, input_width - kernel_width + 1, stride):
                # Обчислення значення на виході через згортку та додавання зміщення
                output_value[d, i // stride, j // stride] = np.sum(
                    padded_input[:, i:i + kernel_height, j:j + kernel_width] * kernel[d]) + bias[d]
    # Повернення вихідного значення
    return output_value

In [ ]:
# Шар максимального пулінгу
def max_pooling(input_value, pool_size=2, stride=2):
    # Визначення розмірів вхідного значення
    input_depth, input_height, input_width = input_value.shape
    # Розрахунок висоти та ширини вихідного значення
    output_height = (input_height - pool_size) // stride + 1
    output_width = (input_width - pool_size) // stride + 1
    # Ініціалізація вихідного значення нулями
    output_value = np.zeros((input_depth, output_height, output_width))

    # Перебір по всіх глибинах вхідного значення
    for d in range(input_depth):
        # Перебір по висоті вхідного значення з урахуванням кроку
        for i in range(0, input_height - pool_size + 1, stride):
            # Перебір по ширині вхідного значення з урахуванням кроку
            for j in range(0, input_width - pool_size + 1, stride):
                # Обчислення максимального значення в поточному підвікні
                output_value[d, i // stride, j // stride] = np.max(
                    input_value[d, i:i + pool_size, j:j + pool_size])
    # Повернення вихідного значення
    return output_value

In [ ]:
# Повнозв'язний шар
def fully_connected_layer(input_value, weights, bias):
    output_value = np.dot(input_value.flatten(), weights) + bias
    return output_value

In [ ]:
# Шар Dropout
def dropout_layer(input_value, dropout_rate):
    dropout_mask = np.random.rand(*input_value.shape) < (1 - dropout_rate)
    return input_value * dropout_mask

In [ ]:
def forward_propagation(image, stride=2, pool_size=2):
    # Встановлення ймовірності dropout
        dropout_rate = 0.25

        # Пропуск через перший шар згортки та функцію активації ReLU
        conv1_output = relu(convolution_layer(image, kernel_conv1, bias_conv1))
        # Застосування Dropout layer
        conv1_output = dropout_layer(conv1_output, dropout_rate)
        # Застосування шару Max Pooling
        pool1_output = max_pooling(conv1_output)

        # Пропуск через другий шар згортки та функцію активації ReLU
        conv2_output = relu(convolution_layer(pool1_output, kernel_conv2, bias_conv2))
        # Застосування Dropout layer
        conv2_output = dropout_layer(conv2_output, dropout_rate)
        # Застосування шару Max Pooling з заданим розміром вікна та кроком
        pool2_output = max_pooling(conv2_output, pool_size=pool_size, stride=stride)

        # Пропуск через третій шар згортки та функцію активації ReLU
        conv3_output = relu(convolution_layer(pool2_output, kernel_conv3, bias_conv3))

        # Пропуск через повнозв'язний шар
        f6_output = fully_connected_layer(conv3_output, weights_f6, bias_f6)
        # Пропуск через шар softmax для отримання ймовірностей класів
        output_value = softmax(fully_connected_layer(f6_output, weights_output, bias_output).reshape(1, -1))

    return output_value

# Recognition test

In [ ]:
# Завантаження бібліотек
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
# Функція, що повертає випадкові зображення з набору даних
def get_random_images(dataset, labels, n):
    num_images = len(dataset)  # Отримання загальної кількості зображень
    # Випадковий вибір індексів без повторень
    random_indices = np.random.choice(num_images, n, replace=False)
    # Вибір випадкових зображень та відповідних міток
    random_images = dataset[random_indices]
    random_labels = labels[random_indices]
    return random_images, random_labels

In [ ]:
# Завантаження набору даних MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Перетворення типу даних зображень на float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Нормалізація зображень шляхом ділення на 255
X_train /= 255
X_test /= 255

In [ ]:
num_cols = 10  # Кількість стовпчиків на графіку
num_images = 10  # Кількість випадкових зображень для відображення

# Отримання випадкових зображень та їх міток з тренувального набору даних
images_array, labels_array = get_random_images(X_train, y_train, num_images)
num_rows = int(np.ceil(num_images / num_cols))  # Кількість рядків на графіку

# Створення фігури та осей для графіку
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 3))

# Цикл по всіх відображуваних зображеннях
for i, ax in enumerate(axes.flatten()):
    if i < num_images:  # Перевірка, чи є ще зображення для відображення
        ax.imshow(images_array[i])  # Відображення зображення на відповідній позиції
        ax.axis('off')  # Вимкнення відображення осей
        ax.text(0.5, -0.20, f"{labels_array[i]}", size=10, ha="center", transform=ax.transAxes)  # Додавання мітки під зображення
    else:
        ax.axis('off')  # Вимкнення відображення осей для порожніх підграфіків

plt.show()  # Відображення графіку

In [ ]:
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # Назви класів

predictions = []  # Список для зберігання передбачень моделі

# Розширення зображень та додавання крайового заповнення
images_array_resize = np.expand_dims(np.pad(images_array, ((0,0), (2,2), (2,2)), mode='constant'), axis=1)

# Цикл по всім зображенням для передбачення
for i_image in range(len(images_array)):
    # Виклик функції forward_propagation для отримання передбачень
    predictions.append(np.argmax(forward_propagation(images_array_resize[i_image])))

# Створення фігури та осей для графіку
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 3))

# Цикл по всім відображуваним зображенням
for i, ax in enumerate(axes.flatten()):
    if i < num_images:  # Перевірка, чи є ще зображення для відображення
        ax.imshow(images_array[i])  # Відображення зображення на відповідній позиції
        ax.axis('off')  # Вимкнення відображення осей
        # Додавання міток під зображення з правильною та передбаченою мітками
        ax.text(0.5, -0.2, f"True: {labels_array[i]}", size=10, ha="center", transform=ax.transAxes)
        ax.text(0.5, -0.4, f"Pred: {class_names[predictions[i]]}", size=10, ha="center", transform=ax.transAxes)
    else:
        ax.axis('off')  # Вимкнення відображення осей для порожніх підграфіків

plt.show()  # Відображення графіку